### Disclaimer:
1) This code only presents the new bias correction methods introduced in the manuscript: **"Variance-Weighted Least Square: A new point-scale bias correction method to improve surface water quality predictions using ensemble machine learning models"**.

2) The associated other steps, such as splitting data into test and train,four ensemble learning model development, hyperparameter tuning using BayesSearchCV(), performance evaluation and residual analysis, application of other exisiting bias correction methods, comprehensive assessment of bias, and visualizations are not demonstarted here as they followed the standard procedures.

In [ ]:
#Necessary Libraries

import numpy as np
import pandas as pd
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold


In [ ]:
#Defining the features, target and the pre-selected hyperparameter.
CFG = {
    "features": [
        'DO(mg/l)', 'Temperature (deg cels)', 'Salinity(ppt)', 'pH',
        'Turbidity(NTU)', 'Phosphate (mg/L)', 'Ammonium (mg/L)',
        'Nitrate+Nitrite (mg/L)', 'TotPAR_max', 'ATemp_max',
        'RH_mean', 'BP_mean', 'Discharge (cfs)'
    ],
    "target": "Chlorophyll-a (ug/L)",
    "rf_base": dict(n_estimators=256, max_depth=20, min_samples_leaf=1, min_samples_split=2 , random_state=42, n_jobs=-1),
    "bins": 10,
    "kf": 5,
    "clip_k": 3.0,
}


In [ ]:
#Loading the training and testing data.
#The splitting was done randomly within specific quartiles by 80%-20% (discussed in the manuscript in detail).
#The splitted data are also provided with the manuscript.

train = pd.read_csv("Training_Data_Chlorophyll_BiasCorrection.csv")
test  = pd.read_csv("Test_Data_Chlorophyll_BiasCorrection.csv")

In [ ]:
#Defining the features and target variables from the loaded data

X_train = train[CFG["features"]].to_numpy()
y_train = train[CFG["target"]].to_numpy().astype(float)

X_test  = test[CFG["features"]].to_numpy()
y_test  = test[CFG["target"]].to_numpy().astype(float)

In [ ]:
#This cell presents the full demonstrattion of RF+VWLS. The code for a six steps procedure as detailed in the manuscript is shown here:


#Step 1) Log transformation of target variable:

y_train_log = np.log1p(np.maximum(y_train, 0.0))



#Step 2) Training a pre-defined baseline RF model with a 5-fold cross validation on the training data (in log pace):

#this model is only for estimating unbiased weights needed for the least square model trained in the subsequent steps)

kf = KFold(n_splits=CFG["kf"], shuffle=True, random_state=42)
yhat_oof_log = np.zeros_like(y_train_log, dtype=float)

for tr_idx, va_idx in kf.split(X_train):
    m = RandomForestRegressor(**CFG["rf_base"])
    m.fit(X_train[tr_idx], y_train_log[tr_idx])
    yhat_oof_log[va_idx] = m.predict(X_train[va_idx])

eps_oof_log = y_train_log - yhat_oof_log #estimating out-of-fold (oof) residuals


#Step 3)Estimating binned (or groupwise) weights for the weighted least square model (VWLS) as the inverse of the out-of-fold residuals:

bins_log = np.quantile(y_train_log, np.linspace(0, 1, CFG["bins"] + 1))
bin_id = np.digitize(y_train_log, bins_log[1:-1], right=True)

var_bins = np.array([
    np.var(eps_oof_log[bin_id == i]) if np.any(bin_id == i) else np.nan
    for i in range(CFG["bins"])
])
var_bins = np.nan_to_num(var_bins, nan=np.nanmedian(var_bins))  #filling empty bins with median variance

w = 1.0 / (var_bins[bin_id] + 1e-6) #estimating weights as the inverse of variance
w /= np.mean(w)  #normalizing

#Step 4) Developing and fitted the variance weighted least square (VWLS):

#this VWLS model simulates residual by taking initial features and base model's predictions as input.
#The VWLS is fitted on the out-of-fold residuals using the groupwise weights obtained in previous step.

Xtr_ext_oof_log = np.column_stack([X_train, yhat_oof_log])
Xtr_gls_oof_log = sm.add_constant(Xtr_ext_oof_log)

vwls = sm.WLS(eps_oof_log, Xtr_gls_oof_log, weights=w).fit() #fitting the VWLS


#Step 5) Re-training the final base RF model and correcting the predictions using VWLS:

#this base RF model is trained on the full training data with same hyperparameters used before,
#the trained RF here is applied to predict for test data (in log-space),
#the fitted VWLS in previous step is the applied to simulate residuals for the test data as additive correction terms
#the simulated residuls are then added back to the base RF predictions to obtain the corrected predictions.

#retraining base RF
rf_log = RandomForestRegressor(**CFG["rf_base"])
rf_log.fit(X_train, y_train_log)
yhat_rf_log_te = rf_log.predict(X_test)


#applying VWLS correction
Xte_ext_log = np.column_stack([X_test, yhat_rf_log_te])
Xte_gls_log = sm.add_constant(Xte_ext_log)
eps_te_log = vwls.predict(Xte_gls_log)

clip = CFG["clip_k"] * np.std(eps_oof_log) #clipping for stability
eps_te_log = np.clip(eps_te_log, -clip, clip)

#correction in log-space
yhat_log_te = yhat_rf_log_te + eps_te_log


#Step 6) Back-transforming the corrected prediction in original scale:
#back-transformation bias is reduced by multiplying a smearing factor which is computed from the out-fold residuals after correction
# ----------------------------
resid_oof_after = eps_oof_log - vwls.predict(Xtr_gls_oof_log)
smear = float(np.mean(np.exp(resid_oof_after))) #smearing fatcor

yhat_vwls = np.expm1(yhat_log_te) * smear

print("VWLS-corrected predictions", yhat_vwls)
